# FlowGenie: Natural-Language Workflow Automation Designer 🧠⚙️

**Track:** Enterprise Agents  
**Tech:** Google ADK (google-adk), Gemini, Multi-Agent, Tools, Evaluation, Sessions

This notebook implements **FlowGenie**, a multi-agent system that turns natural language requests into **structured automation workflows** (JSON). 

**Example:**  
> "When a new lead fills a form, log it in a sheet and send me a summary email."

FlowGenie:
1. **Plans** the workflow (triggers, actions, data flow)  
2. **Evaluates** the quality & safety of the plan  
3. **Simulates execution** of the workflow  
4. Optionally **routes actions to tool agents** (Slack, Sheets, Gmail, Calendar – stubbed)  
5. Returns a structured JSON workflow that a no-code platform (Zapier/IFTTT/Make/etc.) could implement.

This directly targets **enterprise productivity**: automating repetitive workflows without requiring users to write code.


In [1116]:
# 1. Setup & Dependencies

!pip install -q python-dotenv
!pip install -q -U google-adk


In [1117]:

from dotenv import load_dotenv
import os
import json
from typing import Dict, Any, List

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment. Please set it before running.")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

MODEL_NAME = "gemini-2.5-flash-lite"


In [1118]:

import asyncio
from datetime import datetime

from google.genai import types

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search  # just to show built-in tool usage

print("✅ Environment and imports ready. Using model:", MODEL_NAME)


✅ Environment and imports ready. Using model: gemini-2.5-flash-lite


In [1119]:
# Real tools: shared imports & auth helpers

!pip install -q google-api-python-client google-auth google-auth-httplib2 requests


In [1120]:

import base64
from email.mime.text import MIMEText

import requests
from google.oauth2 import service_account
from googleapiclient.discovery import build

USE_REAL_APIS = os.getenv("USE_REAL_APIS", "true").lower() == "true"

def get_service_account_credentials(scopes: List[str]):
    """
    Build service account credentials from GOOGLE_SERVICE_ACCOUNT_JSON.
    Accepts either:
      - a JSON string with double quotes,
      - a path to a JSON file (GOOGLE_SERVICE_ACCOUNT_JSON points to a file),
      - or a Python literal string (fallback via ast.literal_eval).
    If GMAIL_SENDER_EMAIL is set, we also use it as the delegated user
    (for Gmail / Calendar when domain-wide delegation is configured).
    """
    json_str = os.getenv("GOOGLE_SERVICE_ACCOUNT_JSON")
    if not json_str:
        raise RuntimeError("GOOGLE_SERVICE_ACCOUNT_JSON env var not set.")

    # If the env value looks like a path to a file, load it
    if os.path.exists(json_str):
        with open(json_str, "r", encoding="utf-8") as f:
            json_str = f.read()

    # Try strict JSON first, then fallback to ast.literal_eval for Python-style dicts
    try:
        info = json.loads(json_str)
    except Exception:
        try:
            import ast
            info = ast.literal_eval(json_str)
        except Exception as e:
            raise RuntimeError(
                "Failed to parse GOOGLE_SERVICE_ACCOUNT_JSON. "
                "Provide a valid JSON string (double quotes) or a path to a JSON file."
            ) from e

    creds = service_account.Credentials.from_service_account_info(info, scopes=scopes)

    # Optional: domain-wide delegation for Workspace
    delegated_user = os.getenv("GMAIL_SENDER_EMAIL")
    if delegated_user:
        creds = creds.with_subject(delegated_user)

    return creds


In [1121]:
# 2. Workflow Representation

WORKFLOW_JSON_GUIDE = """
You are designing an automation workflow.

Always output a JSON object with this structure:

{
  "name": "<short human-friendly name>",
  "trigger": {
    "type": "<trigger_type>",
    "source": "<where the event originates>",
    "schedule": "<cron or time-based trigger, or null>",
    "conditions": [
      "<optional condition 1>",
      "<optional condition 2>"
    ]
  },
  "actions": [
    {
      "type": "<action_type>",
      "target": "<system or destination>",
      "description": "<what this step does>",
      "inputs": [
        "<critical input or data needed>"
      ]
    }
  ]
}

Rules:
- Use lowercase snake_case for types when possible.
- If something is unknown, put null or an empty list instead of guessing wildly.
- The workflow should be as simple as possible while still solving the user's request.
"""


In [ ]:
# ...existing code...
from google.adk.tools import load_memory

# Simple in-notebook memory store (compatibility layer for tests/debugging).
MEMORY_ENTRIES = {}

def simple_load_memory(payload):
    """
    Accepts payload like {'query': '...'} or a plain string.
    Returns a dict with 'memories' list so planner's load_memory call can find items.
    """
    q = None
    user = globals().get("USER_ID", None)

    if isinstance(payload, dict):
        q = payload.get("query") or payload.get("q") or payload.get("text")
        user = payload.get("user_id") or user
    else:
        q = str(payload)

    entries = MEMORY_ENTRIES.get(user, [])
    if not q:
        return {"memories": entries}

    ql = q.lower().strip()
    matches = [e for e in entries if ql in e.lower()]
    return {"memories": matches}

# Register both the built-in load_memory and the simple compatibility tool.

# ...existing code...

In [1123]:
planner_agent = Agent(
    name="workflow_planner",
    model=MODEL_NAME,
    description="Plans automation workflows from natural-language requests.",
    instruction=(
        "You are an expert workflow automation designer.\n"
        "Given a user's natural-language request, generate exactly ONE automation workflow.\n"
        "DO NOT return a list of workflows.\n"
        "DO NOT return multiple possible interpretations.\n"
        "Return ONLY ONE JSON object that follows this schema:\n"
        f"{WORKFLOW_JSON_GUIDE}\n\n"
        "Respond with ONLY the JSON object. Never return a list or surrounding quotes."
        "If context is unclear or missing, call load_memory() first to retrieve previous details."
    ),
)


In [1124]:
from google.adk.tools import load_memory

planner_agent.tools = [load_memory, simple_load_memory]


In [1125]:
evaluator_agent = Agent(
    name="workflow_evaluator",
    model=MODEL_NAME,
    description="Evaluates workflow quality, safety, and completeness.",
    instruction=(
        "You are evaluating an automation workflow JSON.\n"
        "Check for:\n"
        "- Clarity of trigger\n"
        "- Completeness of actions\n"
        "- Edge cases and failure modes\n"
        "- Privacy or safety concerns\n\n"
        "Return a JSON object with:\n"
        "{\n"
        '  "overall_score": <0-10>,\n'
        '  "verdict": "<ACCEPT or IMPROVE>",\n'
        '  "strengths": ["..."],\n'
        '  "risks": ["..."],\n'
        '  "suggested_changes": ["..."]\n'
        "}\n"
        "Be concise but specific. Respond with ONLY JSON."
        "If referenced entities are unclear (names, IDs, preferences), call load_memory() first to retrieve missing details."
    ),
)


In [1126]:
evaluator_agent.tools = [load_memory, simple_load_memory]

In [ ]:
def simulate_workflow_execution(workflow: Dict[str, Any]) -> Dict[str, Any]:
    """
    Simulate execution of a workflow.

    This does NOT call real APIs, but:
    - iterates through actions,
    - marks each as 'success',
    - returns an execution log.
    """
    name = workflow.get("name", "unnamed_workflow")
    actions = workflow.get("actions", [])

    log = []
    for idx, action in enumerate(actions, start=1):
        log.append(
            {
                "step": idx,
                "type": action.get("type"),
                "target": action.get("target"),
                "status": "success",
                "timestamp": datetime.utcnow().isoformat() + "Z",
                "note": f"Simulated execution of step {idx}",
            }
        )

    return {
        "workflow_name": name,
        "total_steps": len(actions),
        "completed_steps": len(actions),
        "status": "completed" if actions else "no_actions",
        "log": log,
    }


# executor_agent = Agent(
#     name="workflow_executor",
#     model=MODEL_NAME,
#     description="Simulates running automation workflows and validates feasibility.",
#     instruction=(
#         "You are responsible for simulating the execution of a workflow JSON.\n"
#         "ALWAYS call the tool exactly as:\n"
#         "simulate_workflow_execution(workflow=<the full workflow JSON>)\n\n"
#         "After the tool call finishes, return ONLY a JSON object with this structure:\n"
#         "{\n"
#         '  "tool_result": <raw tool result>,\n'
#         '  "summary": "<short human-friendly summary of what the workflow did>"\n'
#         "}\n"
#         "Do not wrap the JSON in quotes, code blocks or backticks."
#     ),
#     tools=[simulate_workflow_execution],
# )


In [1128]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [1129]:
# 3. Real tools (Slack, Sheets, Gmail, Calendar)
# They will call real APIs when USE_REAL_APIS=true and env vars are present,
# otherwise they behave like safe no-ops with clear logs.

# ...existing code...
def slack_send_notification(channel: str, message: str) -> Dict[str, Any]:
    """
    Send a Slack notification via webhook. Returns diagnostic dict on failure.
    Uses SLACK_WEBHOOK_URL from environment.
    """
    import requests
    from urllib.parse import urlparse

    webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not USE_REAL_APIS or not webhook_url:
        print(f"[SLACK NO-OP] Channel={channel} | Message={message}")
        return {
            "channel": channel,
            "message": message,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false or SLACK_WEBHOOK_URL not set",
        }

    # Validate URL has scheme + netloc
    try:
        parsed = urlparse(webhook_url)
        if not parsed.scheme or not parsed.netloc:
            raise ValueError("invalid")
    except Exception:
        return {
            "channel": channel,
            "message": message,
            "status": "error",
            "reason": "SLACK_WEBHOOK_URL is malformed or missing scheme (must start with http:// or https://)",
            "webhook_preview": (webhook_url[:50] + "...") if webhook_url else "<empty>",
        }

    payload = {"text": message}
    try:
        resp = requests.post(webhook_url, json=payload, timeout=10)
    except requests.RequestException as e:
        return {
            "channel": channel,
            "message": message,
            "status": "error",
            "reason": f"request_exception: {type(e).__name__}: {e}",
        }

    return {
        "channel": channel,
        "message": message,
        "http_status": resp.status_code,
        "status": "sent" if resp.status_code in (200, 204) else "error",
        "response_text": resp.text[:200],
    }
# ...existing code...


def sheets_append_row(row_values: List[str]) -> Dict[str, Any]:
    """
    Append a row to Google Sheet.
    spreadsheet_id and sheet_name are loaded from environment.
    """
    # Load from environment
    spreadsheet_id = os.getenv("SHEETS_SPREADSHEET_ID")
    sheet_name = os.getenv("SHEETS_TAB_NAME", "Sheet1")

    if not USE_REAL_APIS:
        print(f"[SHEETS NO-OP] Spreadsheet={spreadsheet_id} | Sheet={sheet_name} | Row={row_values}")
        return {
            "spreadsheet_id": spreadsheet_id,
            "sheet_name": sheet_name,
            "row_values": row_values,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    creds = get_service_account_credentials(scopes)
    service = build("sheets", "v4", credentials=creds)

    range_ = f"{sheet_name}!A1"
    body = {"values": [row_values]}

    result = (
        service.spreadsheets()
        .values()
        .append(
            spreadsheetId=spreadsheet_id,
            range=range_,
            valueInputOption="USER_ENTERED",
            insertDataOption="INSERT_ROWS",
            body=body,
        )
        .execute()
    )

    return {
        "spreadsheet_id": spreadsheet_id,
        "sheet_name": sheet_name,
        "row_values": row_values,
        "status": "appended",
        "updates": result.get("updates", {}),
    }



def gmail_send_email(to: str, subject: str, body: str) -> Dict[str, Any]:
    """
    Send an email using Gmail API and a service account with domain-wide delegation.
    Uses GMAIL_SENDER_EMAIL as 'from' and delegated user.
    """
    sender = os.getenv("GMAIL_SENDER_EMAIL") or "kajal1202patel@gmail.com"

    if not USE_REAL_APIS:
        print(f"[GMAIL NO-OP] From={sender} | To={to} | Subject={subject}")
        return {
            "from": sender,
            "to": to,
            "subject": subject,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/gmail.send"]
    creds = get_service_account_credentials(scopes)
    service = build("gmail", "v1", credentials=creds)

    msg = MIMEText(body)
    msg["to"] = to
    msg["from"] = sender
    msg["subject"] = subject

    raw = base64.urlsafe_b64encode(msg.as_bytes()).decode("utf-8")
    message = {"raw": raw}

    sent = (
        service.users()
        .messages()
        .send(userId="me", body=message)
        .execute()
    )

    return {
        "from": sender,
        "to": to,
        "subject": subject,
        "status": "sent",
        "message_id": sent.get("id"),
    }


def calendar_create_event(calendar_id: str, title: str, start_time: str, end_time: str) -> Dict[str, Any]:
    """
    Create a calendar event using Calendar API.
    Uses CALENDAR_ID from environment by default.
    """
    if not USE_REAL_APIS:
        print(f"[CALENDAR NO-OP] Calendar={calendar_id} | Title={title} | {start_time} -> {end_time}")
        return {
            "calendar_id": calendar_id,
            "title": title,
            "start_time": start_time,
            "end_time": end_time,
            "status": "skipped",
            "reason": "USE_REAL_APIS is false",
        }

    scopes = ["https://www.googleapis.com/auth/calendar"]
    creds = get_service_account_credentials(scopes)
    service = build("calendar", "v3", credentials=creds)

    event = {
        "summary": title,
        "start": {"dateTime": start_time},
        "end": {"dateTime": end_time},
    }

    created = (
        service.events()
        .insert(calendarId=calendar_id, body=event)
        .execute()
    )

    return {
        "calendar_id": calendar_id,
        "title": title,
        "start_time": start_time,
        "end_time": end_time,
        "status": "created",
        "event_id": created.get("id"),
    }


In [1130]:
# # Test sheets_append_row directly
# spreadsheet_id = os.getenv("SHEETS_SPREADSHEET_ID")
# sheet_name = os.getenv("SHEETS_TAB_NAME", "Sheet1")
# row_values = ["0012", "Rajesh", "Login Issue", "Unable to log in"]

# print(f"Testing sheets_append_row with:")
# print(f"  spreadsheet_id: {spreadsheet_id}")
# print(f"  sheet_name: {sheet_name}")
# print(f"  row_values: {row_values}")
# print()

# result = sheets_append_row(spreadsheet_id, sheet_name, row_values)
# print("Result:")
# print(json.dumps(result, indent=2))

In [1131]:
# 4. Tool Agents that wrap the real tools.
# These will be used as "agent tools" by the executor (A2A-style).

slack_agent = Agent(
    name="slack_agent",
    model=MODEL_NAME,
    description="Agent that sends Slack notifications via a tool.",
    instruction=(
        "You send notifications to Slack channels.\n"
        "ALWAYS call the tool slack_send_notification(channel=<channel>, message=<message>).\n"
        "Return ONLY the JSON returned by the tool, no extra text."
    ),
    tools=[slack_send_notification],
)

sheets_agent = Agent(
    name="sheets_agent",
    model=MODEL_NAME,
    description="Agent that appends rows to a Google Sheet via a tool.",
    instruction=(
        "You receive a request to append data to a spreadsheet.\n"
        "Extract ONLY the row_values from the input.\n"
        "ALWAYS call sheets_append_row(row_values=<list of strings>).\n"
        "The spreadsheet_id and sheet_name will be loaded from environment "
        "variables inside the tool.\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[sheets_append_row],
)



gmail_agent = Agent(
    name="gmail_agent",
    model=MODEL_NAME,
    description="Agent that sends emails via a tool.",
    instruction=(
        "You send emails.\n"
        "ALWAYS call gmail_send_email(to=<email>, subject=<subject>, body=<body>).\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[gmail_send_email],
)

calendar_agent = Agent(
    name="calendar_agent",
    model=MODEL_NAME,
    description="Agent that creates calendar events via a tool.",
    instruction=(
        "You create calendar events.\n"
        "ALWAYS call calendar_create_event(calendar_id=<calendar_id>, title=<title>, "
        "start_time=<start>, end_time=<end>).\n"
        "Return ONLY the JSON returned by the tool."
    ),
    tools=[calendar_create_event],
)


In [1132]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import load_memory

# 5. Session + Runners

session_service = InMemorySessionService()
from google.adk.memory import InMemoryMemoryService

memory_service = InMemoryMemoryService()

APP_NAME = "flowgenie"
USER_ID = "demo_user"
SESSION_ID = "flowgenie_session_001"

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={},
)

planner_runner = Runner(
    agent=planner_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service
)

evaluator_runner = Runner(
    agent=evaluator_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service
)

# executor_runner will be created AFTER we redefine executor_agent below.

slack_runner = Runner(
    agent=slack_agent,
    app_name=APP_NAME,
    session_service=session_service
)

sheets_runner = Runner(
    agent=sheets_agent,
    app_name=APP_NAME,
    session_service=session_service
)

gmail_runner = Runner(
    agent=gmail_agent,
    app_name=APP_NAME,
    session_service=session_service
)

calendar_runner = Runner(
    agent=calendar_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("✅ Sessions + runners (planner/evaluator/tool-agents) initialized!")


✅ Sessions + runners (planner/evaluator/tool-agents) initialized!


In [1133]:
executor_agent = Agent(
    name="workflow_executor",
    model=MODEL_NAME,
    description="Analyzes the workflow JSON and creates a structured step-by-step execution plan for tool agents.",
    instruction=(
        "You receive a workflow JSON object with a list of actions.\n"
        "Your job is to produce a structured PLAN listing which tool agent should execute each action.\n\n"
        "For each action in the workflow, extract the input data and create parameters for the tool agent.\n"
        "For sheets: Extract ONLY row values from action.inputs or description and pass as:\n"
        '  "parameters": {"row_values": [<list of strings>]}\n\n'
        "For each action, output an item in the plan like this:\n"
        "{\n"
        '  "action_index": <1-based index>,\n'
        '  "agent": "slack_agent | sheets_agent | gmail_agent | calendar_agent | skipped",\n'
        '  "parameters": {\n'
        '      ... only the essential parameters ...\n'
        '  }\n'
        "}\n\n"
        "Rules for matching:\n"
        "- Slack if type contains 'slack', 'notification', 'alert'\n"
        "- Sheets if type contains 'sheet', 'spreadsheet', 'row'\n"
        "- Gmail if type contains 'email', 'gmail', 'mail'\n"
        "- Calendar if type contains 'calendar', 'event', 'schedule'\n"
        "- Otherwise mark as skipped\n\n"
        "Return ONLY this JSON structure:\n"
        "{\n"
        '  "plan": [... list of tool call definitions ...],\n'
        '  "simulation": {\n'
        '    "workflow_name": "<name>",\n'
        '    "total_steps": <count>,\n'
        '    "completed_steps": <count>,\n'
        '    "status": "completed",\n'
        '    "log": []\n'
        '  },\n'
        '  "summary": "<human friendly short summary>"\n'
        "}\n"
        "Do not wrap response in quotes or code fences. Output ONLY JSON."
    ),
)
# // ...existing code...

In [1134]:
# === Runner for executor agent ===
executor_runner = Runner(
    agent=executor_agent,
    app_name=APP_NAME,
    session_service=session_service
)


In [1135]:
async def execute_action_plan(plan):
    results = []

    for step in plan:
        agent_name = step.get("agent")
        params = step.get("parameters", {})
        idx = step.get("action_index", None)

        if agent_name == "slack_agent":
            resp = await slack_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "sheets_agent":
            resp = await sheets_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "gmail_agent":
            resp = await gmail_runner.run_debug(json.dumps(params), verbose=False)
        elif agent_name == "calendar_agent":
            resp = await calendar_runner.run_debug(json.dumps(params), verbose=False)
        else:
            results.append({
                "action_index": idx,
                "agent": agent_name,
                "result": {"status": "skipped_no_matching_agent"},
            })
            continue

        # Parse the response properly
        try:
            tool_result = parse_json_output(resp)
        except Exception as e:
            tool_result = {"error": str(e), "raw_response": str(resp)}

        results.append({
            "action_index": idx,
            "agent": agent_name,
            "result": tool_result,
        })

    return results

In [1136]:
# ...existing code...
def parse_json_output(raw):
    """Robust JSON extraction for ADK responses."""
    import json
    import re
    from json.decoder import JSONDecodeError

    # If list → try to extract meaningful content from events first
    if isinstance(raw, list):
        # 1) Prefer function_response dicts (tools)
        for ev in raw:
            try:
                parts = getattr(ev, "content").parts
            except Exception:
                parts = None
            if not parts:
                continue
            for part in parts:
                if hasattr(part, "function_response") and part.function_response:
                    resp = getattr(part.function_response, "response", None)
                    if isinstance(resp, dict):
                        return resp
                    # if it's a primitive or list, return as-is
                    return resp
        # 2) Next prefer any text produced by the model
        texts = []
        for ev in raw:
            try:
                parts = getattr(ev, "content").parts
            except Exception:
                parts = None
            if not parts:
                continue
            for part in parts:
                if hasattr(part, "text") and part.text:
                    texts.append(part.text)
        if texts:
            # join multiple text parts into a single string
            return "\n".join(texts)

        # 3) Fallback to trying each item recursively
        for item in raw:
            try:
                return parse_json_output(item)
            except Exception:
                continue

        raise ValueError(f"No valid JSON found in list: {raw}")

    # Handle ADK Event object (single event)
    if hasattr(raw, "content") and hasattr(raw.content, "parts"):
        parts = raw.content.parts
        # If part is a function_response, return its response dict
        for part in parts:
            if hasattr(part, "function_response") and part.function_response:
                resp = getattr(part.function_response, "response", None)
                if isinstance(resp, dict):
                    return resp
                return resp
        # If part has text, use combined text
        texts = [p.text for p in parts if hasattr(p, "text") and p.text]
        if texts:
            raw = "\n".join(texts)
        else:
            # If first part is a function_call (no text), try to use its args as hint
            first = parts[0]
            if hasattr(first, "function_call") and getattr(first, "function_call"):
                # return a representation of the call or empty string
                fc = first.function_call
                try:
                    return getattr(fc, "args", None) or getattr(fc, "payload", None) or ""
                except Exception:
                    return ""

    # Normalize content objects
    elif hasattr(raw, "text"):
        raw = raw.text

    elif hasattr(raw, "parts") and raw.parts:
        raw = raw.parts[0].text

    elif hasattr(raw, "data"):
        raw = raw.data

    # If list (again) → parse first meaningful element
    if isinstance(raw, list):
        for item in raw:
            try:
                return parse_json_output(item)
            except Exception:
                continue
        raise ValueError(f"No valid JSON found in list: {raw}")

    # If dict → return directly
    if isinstance(raw, dict):
        return raw

    if not isinstance(raw, str):
        raise ValueError(f"Unsupported type: {type(raw).__name__}")

    txt = raw.strip()

    # Remove markdown fencing
    if txt.startswith("```"):
        txt = txt.strip("`")
        if txt.startswith("json"):
            parts = txt.split("\n", 1)
            txt = parts[1] if len(parts) > 1 else ""

    # Attempt direct JSON
    try:
        return json.loads(txt)
    except Exception:
        pass

    # Attempt to extract JSON substring
    match = re.search(r"\{[\s\S]*\}", txt)
    if match:
        try:
            return json.loads(match.group(0))
        except Exception:
            pass

    # If still failing, return the raw text instead of raising where appropriate
    # (caller can decide how to handle plain text responses)
    return txt
# ...existing code...

In [1137]:
import re

import re


async def run_flowgenie(user_prompt: str):

    # ---- Memory: detect store or recall intent ----
    store_pattern = r"(remember that|save this|note that|store this)"
    recall_pattern = r"(what did i say earlier|what is my|who did i say|remind me|recall)"

    # FETCH current session
    session = await session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
    )

    # --- Dynamic automatic fact capture (e.g. "my onboarding manager is Rohan") ---
    # BEFORE checking explicit patterns, capture any "my X is Y" statements
    fact_match = re.search(
        r"\bmy\s+([a-zA-Z ]{2,40}?)\s+(?:is|'s|are)\s+([A-Z][\w' \-]{1,80})",
        user_prompt,
        re.IGNORECASE,
    )
    if fact_match:
        subject = fact_match.group(1).strip()
        value = fact_match.group(2).strip()
        entry = f"my {subject} is {value}"
        session.state.setdefault("memory_entries", [])
        session.state["memory_entries"].append(entry)
        # Persist via memory service
        await memory_service.add_session_to_memory(session)
        # Notebook-level compatibility index
        MEMORY_ENTRIES.setdefault(USER_ID, []).append(entry)
        return {
            "memory_saved": True,
            "stored_entry": entry,
            "total_entries": len(session.state["memory_entries"]),
        }

    # 1️⃣ STORE MEMORY (explicit "remember that / note that")
    if re.search(store_pattern, user_prompt, re.IGNORECASE):
        session.state.setdefault("memory_entries", [])
        session.state["memory_entries"].append(user_prompt)

        # Persist via memory service (InMemorySessionService has no save_session())
        await memory_service.add_session_to_memory(session)

        # Also add a cleaned entry to the notebook-level MEMORY_ENTRIES so
        # simple_load_memory/load_memory tools can return it on recall.
        cleaned = re.sub(store_pattern, "", user_prompt, flags=re.IGNORECASE).strip(" .\"'")
        MEMORY_ENTRIES.setdefault(USER_ID, []).append(cleaned)

        return {
            "memory_saved": True,
            "stored_entry": cleaned,
            "total_entries": len(session.state["memory_entries"])
        }

    # 2️⃣ RECALL MEMORY
        # 2️⃣ RECALL MEMORY
    if re.search(recall_pattern, user_prompt, re.IGNORECASE):
        # Use the notebook-level compatibility index
        memories = MEMORY_ENTRIES.get(USER_ID, [])
        
        # Extract the subject being asked about (e.g., "work" from "What is my work?")
        subject_match = re.search(r"(?:what is my|who is my|what did i say about)\s+(\w+)", user_prompt, re.IGNORECASE)
        
        if subject_match and memories:
            subject = subject_match.group(1).lower()
            # Only return memories that mention the specific subject
            matches = [m for m in memories if subject in m.lower()]
            found = matches if matches else ["No memory found for that topic"]
        else:
            # Fallback: return all memories if no specific subject
            found = memories if memories else ["No memory found yet"]
        
        return {
            "memory_recall": found
        }
# ...existing code...
    # Step 1 — Planning
    plan_resp = await planner_runner.run_debug(user_prompt, verbose=True)
    workflow = parse_json_output(plan_resp)

    # Step 2 — Evaluation
    eval_resp = await evaluator_runner.run_debug(json.dumps(workflow), verbose=True)
    evaluation = parse_json_output(eval_resp)

    # Step 3 — Executor generates a tool-call PLAN (not execution yet)
        # Step 3 — Executor generates a tool-call PLAN (not execution yet)
    exec_resp = await executor_runner.run_debug(json.dumps(workflow), verbose=True)
    exec_obj = parse_json_output(exec_resp)

    # Safety: if exec_obj is a string, try to parse it as JSON
    if isinstance(exec_obj, str):
        try:
            exec_obj = json.loads(exec_obj)
        except Exception:
            exec_obj = {"plan": [], "simulation": {}, "summary": exec_obj}

    # Tool call list + simulation results
    action_plan = exec_obj.get("plan", [])
    simulation = exec_obj.get("simulation")

    # Step 4 — Execute tool plan (A2A actions via router)
    tool_results = await execute_action_plan(action_plan)
    
    # Save final session state into long-term memory
    completed = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    await memory_service.add_session_to_memory(completed)

    return {
        "workflow": workflow,
        "evaluation": evaluation,
        "action_plan": action_plan,
        "action_results": tool_results,
        "simulation": simulation,
        "summary": exec_obj.get("summary"),
    }


In [1139]:
await run_flowgenie("I work asa a software engineer")


 ### Created new session: debug_session_id

User > I work asa a software engineer


workflow_planner > [Calling tool: simple_load_memory({'payload': {'query': 'software engineer'}})]
workflow_planner > [Tool result: {'memories': []}]


Event from an unknown agent: workflow_planner, event id: 8bf75887-e72e-4847-85a3-212714c18a4a
Event from an unknown agent: workflow_planner, event id: 0993ac4b-3211-4993-ad44-18ae68ccdbe3
Event from an unknown agent: workflow_planner, event id: 71cdd35f-cfd3-40d5-be59-a883ebfe6f37


workflow_planner > I understand. Is there anything specific you'd like to automate related to your work as a software engineer? For example, are you looking to automate code testing, deployments, or perhaps manage your tasks? Let me know what you have in mind!

 ### Continue session: debug_session_id

User > {"memories": []}


Event from an unknown agent: workflow_evaluator, event id: 8c6e0646-8905-498b-8c56-91046cddd896
Event from an unknown agent: workflow_planner, event id: 8bf75887-e72e-4847-85a3-212714c18a4a
Event from an unknown agent: workflow_planner, event id: 0993ac4b-3211-4993-ad44-18ae68ccdbe3
Event from an unknown agent: workflow_planner, event id: 71cdd35f-cfd3-40d5-be59-a883ebfe6f37


workflow_evaluator > I'm an AI assistant, and I don't have personal experiences or a job title like "software engineer." My purpose is to help you with your tasks by processing information and responding to your requests.

It seems like you might be interacting with a system that has a memory feature. The logs you provided show that the `workflow_evaluator` tool was called to evaluate a workflow. This tool is designed to assess the quality, safety, and completeness of automated workflows.

If you have a specific workflow you'd like me to evaluate, please provide the details! I can help you check for:

*   **Clarity of the trigger:** Is it obvious what starts the workflow?
*   **Completeness of actions:** Does the workflow do everything it's supposed to?
*   **Edge cases and failure modes:** What happens if something unexpected occurs?
*   **Privacy or safety concerns:** Are there any risks involved?

I can then provide a JSON object with an overall score, a verdict (ACCEPT or IMPROVE),

{'workflow': {'memories': []},
 'evaluation': 'I\'m an AI assistant, and I don\'t have personal experiences or a job title like "software engineer." My purpose is to help you with your tasks by processing information and responding to your requests.\n\nIt seems like you might be interacting with a system that has a memory feature. The logs you provided show that the `workflow_evaluator` tool was called to evaluate a workflow. This tool is designed to assess the quality, safety, and completeness of automated workflows.\n\nIf you have a specific workflow you\'d like me to evaluate, please provide the details! I can help you check for:\n\n*   **Clarity of the trigger:** Is it obvious what starts the workflow?\n*   **Completeness of actions:** Does the workflow do everything it\'s supposed to?\n*   **Edge cases and failure modes:** What happens if something unexpected occurs?\n*   **Privacy or safety concerns:** Are there any risks involved?\n\nI can then provide a JSON object with an over

In [1140]:
await run_flowgenie("WHat is my work?")

{'memory_recall': ['No memory found yet']}

In [1141]:
# # --- CORE FLOWGENIE TEST ---

# prompt = """
# add a row to my spreadsheet with:
# Ticket ID 0012, Customer Rajesh, Subject Login Issue, Description Unable to log in,
# and send a Slack notification to #new-channel saying:
# 'New ticket from Rajesh about Login Issue'.
# Also email me a summary.
# """

# result = await run_flowgenie(prompt)

In [1142]:
# NOTE: MCP requires an external MCP tool server; Kaggle cannot host one.
# This stub demonstrates how FlowGenie can be extended with MCP:

"""
from google.adk.tools.mcp_tool import McpToolset

mcp_tools = McpToolset.from_config(
    name="crm_mcp",
    description="Internal CRM ticketing toolset via MCP",
    config_path="mcp/enterprise_crm.json"
)

mcp_agent = Agent(
    name="mcp_agent",
    model=MODEL_NAME,
    instruction="Call MCP tools as needed and summarize results.",
    tools=[mcp_tools],
)
"""
print("📌 MCP extension documented for future deployment.")


📌 MCP extension documented for future deployment.


In [1143]:
pip install aiohttp==3.9.5 --force-reinstall


  Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.8.0-cp312-cp312-win_amd64.whl.metadata (21 kB)
  Using cached multidict-6.7.0-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached yarl-1.22.0-cp312-cp312-win_amd64.whl.metadata (77 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached propcache-0.4.1-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl (369 kB)
Using cached multidict-6.7.0-cp312-cp312-win_amd64.whl (46 kB)
Using cached yarl-1.22.0-cp312-cp312-win_amd64.whl (87 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
Using cached frozenlist-1.8.0-cp312-cp312-win_amd64.whl (44 kB)
Using cached id

In [1144]:
# prompt = "Remember that my favorite CRM system is Slack."
# result = await run_flowgenie(prompt)
# result

In [1145]:
# result = await run_flowgenie("Create an automation workflow using my favorite CRM.")
# print(result)

In [1146]:
# prompt = "Add a row to my spreadsheet with: (0012, Rajesh, Login Issue, Unable to log in)"
# result = await run_flowgenie(prompt)
# print(json.dumps(result, indent=2))

In [1147]:
# prompt = "Send a Slack notification to #new-channel now, saying Thanksss, I'm gratefulll!)"
# result = await run_flowgenie(prompt)
# result

In [1148]:
# prompt = "Send an email to my Gmail saying (FlowGenie test email) with subject (Testing automations)"
# result = await run_flowgenie(prompt)
# result